In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import MeanShift
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from mpl_toolkits.mplot3d import Axes3D

## 1.1 Preprocesamiento de datos

In [ ]:
iris = datasets.load_iris().data

#### 1.b Normalization

In [ ]:
iris_normalized = preprocessing.normalize(iris, norm='l2')

In [ ]:
print("El minimo es %f - El máximo es %f " % (iris_normalized.min(), iris_normalized.max()))

#### 1.c Scaling

##### Escala matriz en un rango [0,1]

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
iris_train_minmax = min_max_scaler.fit_transform(iris)
iris_train_minmax

##### Escala la matriz en un rango [-1,1]

In [ ]:
max_abs_scaler = preprocessing.MaxAbsScaler()
iris_train_maxabs = max_abs_scaler.fit_transform(iris)
print("El minimo es %f - El máximo es %f " % (iris_train_maxabs.min(), iris_train_maxabs.max()))
iris_train_maxabs

#### 2. PCA

In [ ]:
data = pd.read_csv('data/data_otto/train.csv')
data.drop('id', axis=1, inplace=True)
x = data[data.columns[:-1]].values
y = data[data.columns[-1]].values

In [ ]:
for c in [19,28]:
    xp = PCA(n_components=c)
    xp.fit_transform(x)
    print(("La variabilidad del %f%% se explica con %d componentes") % 
            ((xp.explained_variance_ratio_.cumsum()[c-1]*100), c))

In [ ]:
% matplotlib inline

xp = PCA(n_components=2)
xp = x_pca.fit_transform(x)

product_class = np.unique(y)

colors = plt.get_cmap("hsv")

plt.figure(figsize=(10, 4))
for i, p in enumerate(product_class):
    mask = (y == p)
    plt.scatter(xp[mask, 0], xp[mask, 1], 
                c=colors(1. * i / 11), label=p, alpha=0.2)
    
plt.legend(loc="upper left")
plt.xlabel('PC 1')
plt.ylabel('PC 2')

## 1.3 Clustering

#### 3. Mean-Shift sobre el dataset IRIS

In [ ]:
X = datasets.load_iris().data

In [ ]:
ms = MeanShift()
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("Cantidad de clusters : %d" % n_clusters_)

In [ ]:
% matplotlib inline

plt.figure(1)
plt.clf()

colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    my_members = labels == k
    cluster_center = cluster_centers[k]
    plt.plot(X[my_members, 0], X[my_members, 1], col + '.')
    plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)
plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

#### 3. k-means sobre el dataset IRIS

In [ ]:
% matplotlib inline

from sklearn import datasets

np.random.seed(5)

centers = [[1, 1], [-1, -1], [1, -1]]
iris = datasets.load_iris()
X = iris.data
y = iris.target

clustering = KMeans(n_clusters=3)
clustering.fit(X)
labels = clustering.labels_

fig = plt.figure(1)
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

plt.cla()

ax.scatter(X[:, 3], X[:, 0], X[:, 2], c=labels.astype(np.float))

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])
ax.set_xlabel('Petal width')
ax.set_ylabel('Sepal length')
ax.set_zlabel('Petal length')

## 1.4 Clasificación

#### 2. Digits Dataset

In [26]:
digits = datasets.load_digits()

In [27]:
X = digits.data
y = digits.target

In [39]:
clf_linear = SVC(kernel="linear").fit(X, y)
clf_linear.decision_function(X)

/home/leandro/virtualenvs/DSS/lib/python3.4/site-packages/sklearn/svm/base.py:547: ChangedBehaviorWarning: The decision_function_shape default value will change from 'ovo' to 'ovr' in 0.18. This will change the shape of the decision function returned by SVC.
  "SVC.", ChangedBehaviorWarning)


array([[ 1.88793344,  1.72479957,  1.48441384, ...,  0.21161172,
         0.09035082, -0.16607813],
       [-2.50834984, -1.26755844, -1.49864758, ..., -2.10079496,
        -0.18982678,  3.35797939],
       [-1.45631393, -1.4573753 , -0.9824458 , ..., -2.30160695,
         1.47109128,  5.36545509],
       ..., 
       [-0.73699949, -0.71192053, -0.80932215, ..., -2.91786091,
        -1.24556113,  6.34912842],
       [-0.1277185 ,  0.59877042, -0.32336706, ..., -1.04836566,
        -2.89548762, -2.22455885],
       [-0.29648248, -0.42832547, -0.93659351, ..., -2.57076768,
        -2.33215878,  3.43238452]])

In [40]:
clf_rbf = SVC(kernel="rbf").fit(X, y)
clf_linear.decision_function(X)

/home/leandro/virtualenvs/DSS/lib/python3.4/site-packages/sklearn/svm/base.py:547: ChangedBehaviorWarning: The decision_function_shape default value will change from 'ovo' to 'ovr' in 0.18. This will change the shape of the decision function returned by SVC.
  "SVC.", ChangedBehaviorWarning)


array([[ 1.88793344,  1.72479957,  1.48441384, ...,  0.21161172,
         0.09035082, -0.16607813],
       [-2.50834984, -1.26755844, -1.49864758, ..., -2.10079496,
        -0.18982678,  3.35797939],
       [-1.45631393, -1.4573753 , -0.9824458 , ..., -2.30160695,
         1.47109128,  5.36545509],
       ..., 
       [-0.73699949, -0.71192053, -0.80932215, ..., -2.91786091,
        -1.24556113,  6.34912842],
       [-0.1277185 ,  0.59877042, -0.32336706, ..., -1.04836566,
        -2.89548762, -2.22455885],
       [-0.29648248, -0.42832547, -0.93659351, ..., -2.57076768,
        -2.33215878,  3.43238452]])

#### 3. Vinos

In [2]:
data = pd.read_csv('data/winequality-red.csv', sep=";")
X = data[data.columns[:-1]].as_matrix()
y = data["quality"].as_matrix()

In [8]:
clf_linear = SVC(kernel='linear').fit(X, y)

In [11]:
clf_linear.decision_function(X)

/home/leandro/virtualenvs/DSS/lib/python3.4/site-packages/sklearn/svm/base.py:547: ChangedBehaviorWarning: The decision_function_shape default value will change from 'ovo' to 'ovr' in 0.18. This will change the shape of the decision function returned by SVC.
  "SVC.", ChangedBehaviorWarning)


array([[-1.06579787, -1.00061273, -1.05658602, ...,  1.00683431,
         1.00124289,  1.00112503],
       [-1.19036218, -1.00195253, -1.22230927, ...,  1.00711328,
         1.00106702,  1.00107153],
       [-1.1894284 , -1.00182166, -1.23898964, ...,  1.00584657,
         1.0010491 ,  1.00119803],
       ..., 
       [-1.25610475, -1.00116715, -1.23969461, ...,  1.00186007,
         1.0007635 ,  1.00051433],
       [-1.13749881, -1.00053369, -1.09787689, ...,  1.00561539,
         1.00098968,  1.00013995],
       [-1.28076213, -1.00185758, -1.40607796, ...,  1.00034108,
         1.0007179 ,  1.00013949]])

In [9]:
clf_rbf = SVC(kernel='rbf').fit(X, y)

In [21]:
clf_rbf.decision_function(X)

/home/leandro/virtualenvs/DSS/lib/python3.4/site-packages/sklearn/svm/base.py:547: ChangedBehaviorWarning: The decision_function_shape default value will change from 'ovo' to 'ovr' in 0.18. This will change the shape of the decision function returned by SVC.
  "SVC.", ChangedBehaviorWarning)


array([[-0.9253644 , -1.04149227, -1.00539788, ...,  0.92105959,
         1.03051485,  1.02028612],
       [-0.88143482, -0.99988388, -1.00001641, ...,  0.99920748,
         1.00011079,  0.89518142],
       [-0.97400674, -1.01269286, -1.0013847 , ...,  0.82172435,
         1.00330028,  0.9271674 ],
       ..., 
       [-0.87714904, -0.99713674, -1.00004743, ...,  1.05011128,
         1.00679818,  0.89708   ],
       [-0.85839131, -1.01276167, -1.07805337, ...,  1.15812344,
         1.113498  ,  0.70129747],
       [-0.92100529, -1.0023291 , -1.00002434, ...,  1.00032841,
         1.02753355,  0.98328321]])